## Transfer learning

In this notebook we use **pre-trained** neural networks as a starting point for our own models. Other people have trained enormous general-purpose CNNs for image classification. We can exploit the features that these CNNs have found for our own image classification problem.

Typically, transfer learning works by loading a pre-trained network and removing the final layer (which predicts class membership), since this will be particular to the problem used to train the original network. We then add on our own final layer, which contains our own output nodes. We then retrain the network (either just the final layer, or the whole network). 

In [1]:
Sys.setenv(KERAS_BACKEND = "tensorflow")
library(keras)

Say where the data is...

In [4]:
train_directory <- "data/invasives/train/"
validation_directory <- "data/invasives/validation/"
test_directory <- "data/invasives/test/"

And work out how many images we have.

In [5]:
train_samples <- length(list.files(paste(train_directory,"invasive",sep=""))) +
    length(list.files(paste(train_directory,"non_invasive",sep="")))

validation_samples <- length(list.files(paste(validation_directory,"invasive",sep=""))) +
    length(list.files(paste(validation_directory,"non_invasive",sep="")))

test_samples <- length(list.files(paste(test_directory,"invasive",sep=""))) +
    length(list.files(paste(test_directory,"non_invasive",sep="")))

In [6]:
train_samples

[1] 953

In this case we will use the VGG16 pre-trained network. This network needs 224x224 images, so we set desired image height and width accordingly.

In [12]:
img_height <- 224
img_width <- 224
batch_size <- 16
epochs <- 4

### Data generators

In [19]:
train_generator <- flow_images_from_directory(
  train_directory, 
  generator = image_data_generator(),
  target_size = c(img_height, img_width),
  color_mode = "rgb",
  class_mode = "binary", 
  batch_size = batch_size, 
  shuffle = TRUE,
  seed = 123)

validation_generator <- flow_images_from_directory(
  validation_directory, 
  generator = image_data_generator(), 
  target_size = c(img_height, img_width), 
  color_mode = "rgb", 
  classes = NULL,
  class_mode = "binary", 
  batch_size = batch_size, 
  shuffle = TRUE,
  seed = 123)

test_generator <- flow_images_from_directory(
  test_directory, 
  generator = image_data_generator(),
  target_size = c(img_height, img_width), 
  color_mode = "rgb", 
  class_mode = "binary", 
  batch_size = 1,
  shuffle = FALSE) 

### Loading pre-trained model and adding custom layers 

In [9]:
base_model <- application_vgg16(weights = "imagenet", 
                                       include_top = FALSE)

### Add our custom layers

In [10]:
predictions <- base_model$output %>% 
  layer_global_average_pooling_2d() %>% 
  layer_dense(units = 1024, activation = "relu") %>% 
  layer_dense(units = 1, activation = "sigmoid")

model <- keras_model(inputs = base_model$input, 
                     outputs = predictions)

### Compile the model

In [11]:
model %>% compile(
  loss = "binary_crossentropy",
  optimizer = optimizer_sgd(lr = 0.0001, 
                            momentum = 0.9, 
                            decay = 1e-5),
  metrics = "accuracy"
)

### Fit the model

In [14]:
model %>% fit_generator(
  train_generator,
  steps_per_epoch = 1, 
  epochs = 1, 
  validation_data = validation_generator,
  validation_steps = 1,
  verbose = 1)

<keras.callbacks.History>

### Evaluate the model

In [18]:
model

Model
________________________________________________________________________________
Layer (type)                        Output Shape                    Param #     
input_1 (InputLayer)                (None, None, None, 3)           0           
________________________________________________________________________________
block1_conv1 (Conv2D)               (None, None, None, 64)          1792        
________________________________________________________________________________
block1_conv2 (Conv2D)               (None, None, None, 64)          36928       
________________________________________________________________________________
block1_pool (MaxPooling2D)          (None, None, None, 64)          0           
________________________________________________________________________________
block2_conv1 (Conv2D)               (None, None, None, 128)         73856       
________________________________________________________________________________
block2_conv2 (Conv2D) 

In [25]:
model %>% evaluate_generator(model, test_generator, steps = 1,
                            max_queue_size = 1)

ERROR: Error in evaluate_generator(., model, test_generator, steps = 1, max_queue_size = 1): unused argument (test_generator)


In [16]:
model %>% predict_generator(model,
                           test_generator,
                           steps = 1)

ERROR: Error in as.integer(max_queue_size): cannot coerce type 'environment' to vector of type 'integer'
